@brief py -> ipynb to resolve wandb error (failed)  
@note  upload IBM_Debater_(R)_CE-EMNLP-2015.v3/*_preprocess-02_awk.txt in content/  
@note  change runtime to gpu
@note  set wandb api key after !wandb login

In [1]:
!pip install transformers
!pip install simpletransformers
!wandb login [WRITE-API-KEY]

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 43.4 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 61 kB 477 kB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 247 kB 5.2 MB/s 
     |████████████████████████████████| 298 kB 38.7 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 1.7 MB 36.2 MB/s 
     |████████████████████████████████| 1.2 MB 34.6 MB/s 
     |████████████████████████████████| 9.1 MB 43.8 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 180 kB 44.6 MB/s 
     |████████████████████████████████| 140 kB 41.3 MB/s 
     |████████████████████████████████| 63 kB 1

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
class Log:
    __ENABLE_DEBUG = True
    # __ENABLE_DEBUG = False
    __ENABLE_VERBOSE = True
    # __ENABLE_VERBOSE = False

    __RESET = "\033[0m"
    __RED = "\033[41m"
    __YELLOW = "\033[33m"

    @staticmethod
    def d(*args):
        if Log.__ENABLE_DEBUG:
            print(*args)

    # VERBOSE
    @staticmethod
    def v(*args):
        if Log.__ENABLE_VERBOSE:
            print(*args)

    # ERROR
    @staticmethod
    def e(*args):
        print(Log.__RED, *args, Log.__RESET, file=stderr)

    # WARNING
    @staticmethod
    def w(*args):
        print(Log.__YELLOW, *args, Log.__RESET)

In [4]:
# -*- coding: utf-8 -*-
"""process_01_optimize_classifier_as_claim_or_evidence.py

@author    Kataoka Nagi (calm1836[at]gmail.com)
@brief     optimize transformer txt classifier as claim or evidence
@note      [sent-1#sent-2#...\n, ...] -> [e;[ec-score-array];sent-1#c;[ec-score-array];sent-2...\n, ...]
@note      python3 process_01_train_classifier_as_claim_or_evidence.py
@date      2021-12-04 06:20:06
@version   1.0
@history   add
@see       transformer_classifier_as_claim_or_evidence.ipynb
@see       [hands-on-ml 16章 Sentiment Analysys](https://github.com/ageron/handson-ml2/blob/master/16_nlp_with_rnns_and_attention.ipynb)
@copyright This file includes the work that is distributed in the [Apache License 2.0](http://www.apache.org/licenses/LICENSE-2.0)
@copyright This file includes the work that is distributed in the [CC BY-CA 3.0](https://creativecommons.org/licenses/by-sa/3.0/)
@copyright (c) 2021 Kataoka Nagi This src is released under the Apache License 2.0 & CC BY-CA 3.0, see LICENSE.

"""

# from utils.log import Log as log
from sys import stderr
import numpy as np
import random
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
import time
import wandb
from sklearn.metrics import accuracy_score

log = Log()

CLAIMS_DIR = "IBM_Debater_(R)_CE-EMNLP-2015.v3/claims_preprocess-02_awk.txt"
EVIDENCE_DIR = "IBM_Debater_(R)_CE-EMNLP-2015.v3/evidence_preprocess-02_awk.txt"

CLAIMS_LABEL = 1
EVIDENCE_LABEL = 0

SHUFFLE_SEED = 2021
EVAL_RATE = 0.2

NUM_EPOCHS = 10
MODEL_SEED = 2021
CLASSIFICATION_MODEL_TYPE = 'roberta'
CLASSIFICATION_MODEL_NAME = 'roberta-base'


def train():
    # wandbの初期化
    wandb.init()

    # モデルの作成
    model = ClassificationModel(
        model_type=CLASSIFICATION_MODEL_TYPE,
        model_name=CLASSIFICATION_MODEL_NAME,
        use_cuda=True,
        args=model_args,
        sweep_config=wandb.config)

    # 学習
    model.train_model(train_df,
                      eval_df=eval_df,
                      accuracy=lambda truth,
                      predictions: accuracy_score(truth,
                                                  [round(p) for p in predictions]),
                      overwrite_output_dir=True)

    # wandbのログ保存
    wandb.log(model.results)

    # wandbの同期
    wandb.join()


# def main():
claims = []
evidences = []

# load
log.d("*** load data of claim & evidence***")

# import claim data
with open(CLAIMS_DIR, "r", encoding="utf_8") as f:
    sentences = f.readlines()
    log.v("claims.txt lines:")
    log.v(sentences[0])
    log.v(sentences[1])
    log.v(sentences[2])
    log.v()
    for sentence in sentences:
        claims.append(sentence.rstrip("\n"))
    if claims[-1] is None or len(claims[-1]) == 0:
        claims.pop(-1)

# import evidence data
with open(EVIDENCE_DIR, "r", encoding="utf_8") as f:
    sentences = f.readlines()
    log.v("evidence.txt lines:")
    log.v(sentences[0])
    log.v(sentences[1])
    log.v(sentences[2])
    log.v()
    for sentence in sentences:
        evidences.append(sentence.rstrip("\n"))
    if evidences[-1] is None or len(evidences[-1]) == 0:
        evidences.pop(-1)

# print debug
log.v("claims:")
log.v(claims[0])
log.v(claims[1])
log.v()

log.v("evidence:")
log.v(evidences[0])
log.v(evidences[1])

"""### ラベリング"""
log.d("*** labeling claim & evidence ***")
log.v("CLAIMS_LABEL:", CLAIMS_LABEL)
log.v("EVIDENCE_LABEL:", EVIDENCE_LABEL)

if (CLAIMS_LABEL):
    claims_labels = np.ones(len(claims)).tolist()
else:
    claims_labels = np.zeros(len(claims)).tolist()

if (EVIDENCE_LABEL):
    evidence_labels = np.ones(len(evidences)).tolist()
else:
    evidence_labels = np.zeros(len(evidences)).tolist()

log.v(claims_labels[0])
log.v(evidence_labels[0])

claims_dataset = []
evidence_dataset = []

for idx in range(len(claims)):
    claims_dataset.append([claims[idx], claims_labels[idx]])

for idx in range(len(evidences)):
    evidence_dataset.append([evidences[idx], evidence_labels[idx]])

log.v(claims_dataset[0])
log.v(evidence_dataset[0])

"""### 結合"""
log.d("*** cat 2 datasets ***")

dataset = claims_dataset + evidence_dataset
log.v(dataset[0])
log.v(dataset[-1])

"""### シャッフル"""
log.d("*** shuffle dataset order ***")

log.d("SHUFFLE_SEED:", SHUFFLE_SEED)
random.seed(SHUFFLE_SEED)

random.shuffle(dataset)
log.v(dataset[0])
log.v(dataset[-1])

"""### 訓練、評価~~、テストデータ~~に分割"""
log.d("*** split dataset to eval & train ***")
log.d("EVAL_RATE:", EVAL_RATE)

dataset_num = len(dataset)
eval_num = int(dataset_num * EVAL_RATE)
train_num = dataset_num - eval_num

eval_dataset = dataset[:eval_num]
train_dataset = dataset[eval_num:]

log.v("dataset_num: ", dataset_num)
log.v("eval_num: ", eval_num)
log.v("train_num: ", train_num)

"""## 分類器の作成
- [Simple Transformers 入門 (1) - テキスト分類 ](https://note.com/npaka/n/nfe2436ea5301#9VV1a)
- [Simple Transformers 入門 (10) - ハイパーパラメータの最適化 ](https://note.com/npaka/n/n298f269c2275)

# Simple Transformerのインストール
"""

# !pip install transformers
# !pip install simpletransformers

"""### モデルの評価まで"""
log.d("*** train & evaluate model ***")

# Sweepsの設定
#! @see [Simple Transformers 入門 (10) - ハイパーパラメータの最適化](https://note.com/npaka/n/n298f269c2275)
sweep_config = {
    "name": "vanilla-sweep-batch-8",
    "method": "bayes",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "parameters": {
        "num_train_epochs": {"min": 1, "max": 10},
        "learning_rate": {"min": float(0), "max": float(4e-4)},
    },
    "early_terminate": {"type": "hyperband", "min_iter": 6, },
}
sweep_id = wandb.sweep(
    sweep_config,
    project="RTE - Hyperparameter Optimization")

# ログの設定
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# 学習データの作成
train_df = pd.DataFrame(train_dataset)

# 評価データの作成
eval_df = pd.DataFrame(eval_dataset)

# モデルの作成

log.d("NUM_EPOCHS:", NUM_EPOCHS)
log.d("MODEL_SEED:", MODEL_SEED)
model_args = ClassificationArgs()
model_args.num_train_epochs = NUM_EPOCHS
# model_args.reprocess_input_data = True
model_args.manual_seed = MODEL_SEED

#! @see [Simple Transformers 入門 (10) - ハイパーパラメータの最適化](https://note.com/npaka/n/n298f269c2275)
model_args.learning_rate = float(1e-5)  # 学習率
model_args.train_batch_size = 8  # 学習のバッチサイズ
model_args.eval_batch_size = 16  # 評価のバッチサイズ
model_args.evaluate_during_training = True
model_args.labels_list = ["not_entailment", "entailment"]

log.d("CLASSIFICATION_MODEL_TYPE:", CLASSIFICATION_MODEL_TYPE)
log.d("CLASSIFICATION_MODEL_NAME:", CLASSIFICATION_MODEL_NAME)

# !rm - rf outputs/

# 学習
start_time = time.time()
# wandbで学習
wandb.agent(sweep_id, train)
log.d("train time (sec):", time.time() - start_time)

# 評価
# result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# スコア計算
# result, model_outputs, wrong_predictions = model.eval_model(
#     eval_df, acc=sklearn.metrics.accuracy_score)

# if __name__ == "__main__":
#     main()


*** load data of claim & evidence***
claims.txt lines:
exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world.

video game violence is not related to serious aggressive behavior in real life.

some violent video games may actually have a prosocial effect in some contexts.


evidence.txt lines:
a 2001 study found that exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world.

the most recent large scale metaanlysis examining 130 studies with over 130000 subjects worldwide concluded that exposure to violent video games causes both short term and long term aggression in players.

one study did find an increase in reports of bullying noting our research found that certain patterns of video game play were much more likely to be associated with these types of behavioral problems.


claims:
exposure to vio

INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=None, count=None


Create sweep with ID: 0y20m7fs
Sweep URL: https://wandb.ai/kataoka-nagi/RTE%20-%20Hyperparameter%20Optimization/sweeps/0y20m7fs
NUM_EPOCHS: 10
MODEL_SEED: 2021
CLASSIFICATION_MODEL_TYPE: roberta
CLASSIFICATION_MODEL_NAME: roberta-base


wandb: Agent Starting Run: rl6kuziw with config:
wandb: 	learning_rate: 0.0002507719460497503
wandb: 	num_train_epochs: 5
wandb: Currently logged in as: kataoka-nagi (use `wandb login --relogin` to force relogin)


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

Run 2b7bovjg errored: KeyError(1.0)
wandb: ERROR Run 2b7bovjg errored: KeyError(1.0)
wandb: Agent Starting Run: ls46yuzf with config:
wandb: 	learning_rate: 6.279038242219187e-05
wandb: 	num_train_epochs: 3


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

Run ls46yuzf errored: KeyError(1.0)
wandb: ERROR Run ls46yuzf errored: KeyError(1.0)
Detected 5 failed runs in a row at start, killing sweep.
wandb: ERROR Detected 5 failed runs in a row at start, killing sweep.
wandb: To change this value set WANDB_AGENT_MAX_INITIAL_FAILURES=val


train time (sec): 119.80763411521912
